# **<font color='crimson'>«SQLite & Google Colab. Анализ данных по городам России»</font>**

---


# <font color='teal'>**1 Подготовка базы данных для работы**</font>

---

### <font color='navy'>**1.1 Импорт библиотек. Подготовка датасета для загрузки в базу данных**</font>

---

In [ ]:
# основные библиотеки
import sqlite3
import gdown
import pandas as pd

In [ ]:
# проверим версию библиотеки sqlite3
sqlite3.sqlite_version

'3.37.2'

In [ ]:
# проверяем версию gdown
gdown.__version__

'4.7.3'

In [ ]:
# устанавливаем более раннюю версию gdown
!pip install -q gdown==4.6.0

In [ ]:
# загружаем csv-файл для последующей
# загрузки в базу данных
!gdown 1qGVn7l-n_Lh0zGpGV36M3I8q6zR39CBi

Downloading...
From: https://drive.google.com/uc?id=1qGVn7l-n_Lh0zGpGV36M3I8q6zR39CBi
To: /content/city.csv
100% 324k/324k [00:00<00:00, 86.9MB/s]


In [ ]:
# сохраняем csv-файл в переменную
city = pd.read_csv('/content/city.csv')

In [ ]:
# выведем первые пять строк датасета city
city.head()

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,fias_level,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year
0,"Респ Адыгея, г Адыгейск",385200.0,Россия,Южный,Респ,Адыгея,NaN,NaN,г,Адыгейск,...,4,0,79403000000,79703000001,107,UTC+3,44.878414,39.190289,12689,1969
1,г Майкоп,385000.0,Россия,Южный,Респ,Адыгея,NaN,NaN,г,Майкоп,...,4,2,79401000000,79701000001,105,UTC+3,44.609827,40.100661,144055,1857
2,г Горно-Алтайск,649000.0,Россия,Сибирский,Респ,Алтай,NaN,NaN,г,Горно-Алтайск,...,4,2,84401000000,84701000,400,UTC+7,51.958103,85.960324,62861,1830
3,"Алтайский край, г Алейск",658125.0,Россия,Сибирский,край,Алтайский,NaN,NaN,г,Алейск,...,4,0,1403000000,1703000,2201,UTC+7,52.492251,82.779361,28528,1913
4,г Барнаул,656000.0,Россия,Сибирский,край,Алтайский,NaN,NaN,г,Барнаул,...,4,2,1401000000,1701000,2200,UTC+7,53.347997,83.779806,635585,1730


### <font color='navy'>**1.2 Подготовка базы данных для отправки запросов с помощью SQLite**</font>

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# на Google Drive создаем файл для хранения базы данных
# с возможностью переиспользования
con = sqlite3.connect(
    '/content/drive/MyDrive/Colab Notebooks/SQL/city.db')

In [ ]:
# заливаем таблицу city в базу данных
city.to_sql(
    'city',
    con,
    index = False,
    if_exists = 'replace'
)

1117

Для автоматизации написания запросов удобным инструментом является функция.

In [ ]:
# задаем функцию для написания запросов
def select(sql):
    return pd.read_sql(sql, con)

In [ ]:
# создадим запрос для проверки загруженной базы city
sql = '''
    SELECT
        COUNT(DISTINCT t.federal_district) AS count_districts
        ,COUNT(DISTINCT t.region) AS count_regions
        ,COUNT(DISTINCT t.city) AS count_cities
    FROM
        city AS t;
'''

In [ ]:
select(sql)

,count_districts,count_regions,count_cities
0,8,85,1092


Базы данных корректно загружены и готовы для отправки запросов с помощью **SQLite**.

# <font color='teal'>**2 Анализ данных в Google Colab с помощью SQLite**</font>

---

In [ ]:
###

Найти города с самым длинным названием (вывести названия городов и длину названий городов).

In [ ]:
sql = '''
--найдем максимальную длину названия города
WITH max_length_name AS (
    SELECT
        MAX(length(t.city)) AS max_length
    FROM
        city AS t
)
--выведем названия городов с самым большим числом символов
SELECT
    t.city
    ,length(t.city) AS name_length
FROM
    city AS t
WHERE
    length(t.city) = (
        SELECT t.max_length
        FROM max_length_name AS t
    );
'''

In [ ]:
select(sql)

,city,name_length
0,Александровск-Сахалинский,25


In [ ]:
###

Посчитать количество городов для каждого часового пояса в Сибирском и Приволжском федеральных округах. Вывести столбцы **timezone** и **city_count**, отсортировать по значению часового пояса.

Указать в ответе значение **city_count** для **timezone = UTC+5**.

In [ ]:
sql = '''
    SELECT
        t.timezone
        ,COUNT(1) AS city_count
    FROM
        city AS t
    WHERE
        t.federal_district IN ('Сибирский', 'Приволжский')
    GROUP BY
        t.timezone
    ORDER BY
        t.timezone;
'''

In [ ]:
select(sql)

,timezone,city_count
0,UTC+3,101
1,UTC+4,41
2,UTC+5,58
3,UTC+6,6
4,UTC+7,86
5,UTC+8,22


In [ ]:
# выведем число городов в чаосвом поясе 'UTC+5'
sql = '''
    SELECT
        t.timezone
        ,COUNT(1) AS city_count
    FROM
        city AS t
    WHERE
        t.federal_district IN ('Сибирский', 'Приволжский')
        AND t.timezone  LIKE 'UTC+5'
    GROUP BY
        t.timezone
    ORDER BY
        t.timezone;
'''

In [ ]:
select(sql)

,timezone,city_count
0,UTC+5,58


In [ ]:
###

**Найти три ближайших к Самаре города, не считая саму Самару.**

In [ ]:
# получим координаты Самары
sql = '''
    SELECT
        t.geo_lat AS samara_geo_lat
        ,t.geo_lon AS samara_geo_lon
    FROM
        city AS t
    WHERE
        t.city = 'Самара';
'''

In [ ]:
select(sql)

,samara_geo_lat,samara_geo_lon
0,53.195031,50.106952


In [ ]:
sql = '''
    --получим координаты Самары
    WITH samara_location AS (
        SELECT
            t.geo_lat AS samara_geo_lat
            ,t.geo_lon AS samara_geo_lon
        FROM
            city AS t
        WHERE
            t.city = 'Самара'
    )
-- вычислим расстояниe от Самары до других городов
SELECT
    t.city
    ,SQRT(
        POWER((
            (SELECT t.samara_geo_lat FROM samara_location AS t )
            - t.geo_lat), 2)
        + POWER(((SELECT t.samara_geo_lon FROM samara_location AS t )
            - t.geo_lon), 2)) AS distance
FROM
    city AS t
WHERE
    SQRT(
        POWER((
            (SELECT t.samara_geo_lat FROM samara_location AS t )
            - t.geo_lat), 2)
        + POWER(((SELECT t.samara_geo_lon FROM samara_location AS t )
            - t.geo_lon), 2)) <> 0
ORDER BY
    distance ASC
LIMIT 3;
'''

In [ ]:
select(sql)

,city,distance
0,Новокуйбышевск,0.185697
1,Чапаевск,0.358069
2,Кинель,0.528066


In [ ]:
###

Вычислить число городов в каждом федеральном округе.

In [ ]:
sql = '''
    SELECT
        t.timezone
        ,COUNT(1) AS city_count
    FROM
        city AS t
    GROUP BY
        t.timezone
    ORDER BY
        COUNT(1) DESC;
'''

In [ ]:
select(sql)

,timezone,city_count
0,UTC+3,660
1,UTC+5,173
2,UTC+7,86
3,UTC+4,66
4,UTC+9,31
5,UTC+8,28
6,UTC+2,22
7,UTC+10,22
8,UTC+11,17
9,UTC+6,6


In [ ]:
###

Вывести тип данных для ячеек с названиями городов, адресов, коррдинат городов Томской области.

In [ ]:
sql = '''
    SELECT
        t.city
        ,TYPEOF(t.city) AS city_type
        ,t.address
        ,TYPEOF(t.address) AS address_type
        ,t.geo_lat
        ,TYPEOF(t.geo_lat) AS type_geo_lat
        ,t.geo_lon
        ,TYPEOF(t.geo_lon) AS type_geo_lon
    FROM
        city AS t
    WHERE
        t.region LIKE 'Томская';
'''

In [ ]:
select(sql)

,city,city_type,address,address_type,geo_lat,type_geo_lat,geo_lon,type_geo_lon
0,Асино,text,"Томская обл, г Асино",text,56.990708,real,86.176526,real
1,Кедровый,text,"Томская обл, г Кедровый",text,57.561869,real,79.567782,real
2,Колпашево,text,"Томская обл, г Колпашево",text,58.311425,real,82.902583,real
3,Северск,text,"Томская обл, г Северск",text,56.603128,real,84.880993,real
4,Стрежевой,text,"Томская обл, г Стрежевой",text,60.732895,real,77.604122,real
5,Томск,text,г Томск,text,56.484580,real,84.948158,real


In [ ]:
###

Создать данные разных типов в одном признаке новой таблицы.

In [ ]:
sql = '''
    WITH subquery AS (
        SELECT 42 AS value
        UNION ALL
        SELECT 3.14
        UNION ALL
        SELECT 'SQL pour tous!'
        UNION ALL
        SELECT x'1313'
        UNION ALL
        SELECT NULL
    )
SELECT
    t.value
    ,TYPEOF(t.value) AS type
FROM
    subquery AS t;
'''

In [ ]:
select(sql)

,value,type
0,42,integer
1,3.14,real
2,SQL pour tous!,text
3,b'\x13\x13',blob
4,None,null


In [ ]:
###

Изменить типы данных.

In [ ]:
sql = '''
    SELECT 'CAST(42 AS REAL) = ' || CAST(42 AS REAL) AS change_type
    UNION ALL
    SELECT 'CAST(42 AS TEXT) = ' || CAST(42 AS TEXT)
    UNION ALL
    SELECT 'CAST(3.12 AS INTEGER) = ' || CAST(3.14 AS INTEGER)
    UNION ALL
    SELECT 'CAST(3.14 AS TEXT) = ' || CAST(3.14 AS TEXT)
    UNION ALL
    SELECT 'CAST("SQL pour tous!" AS INTEGER) = '
        || CAST('SQL pour tous!' AS INTEGER)
    UNION ALL
    SELECT 'CAST("12. SQL pour tous!" AS INTEGER) = '
        || CAST('12. SQL pour tous!' AS INTEGER)
    UNION ALL
    SELECT 'CAST(NULL AS REAL) = ' || CAST(NULL AS REAL);
'''

In [ ]:
select(sql)

,change_type
0,CAST(42 AS REAL) = 42.0
1,CAST(42 AS TEXT) = 42
2,CAST(3.12 AS INTEGER) = 3
3,CAST(3.14 AS TEXT) = 3.14
4,"CAST(""SQL pour tous!"" AS INTEGER) = 0"
5,"CAST(""12. SQL pour tous!"" AS INTEGER) = 12"
6,None


In [ ]:
###

In [ ]:
sql = '''
    SELECT MAX(ROUND(42), ROUND(42.3), ROUND(42.5))
'''

In [ ]:
select(sql)

,"MAX(ROUND(42), ROUND(42.3), ROUND(42.5))"
0,43.0


In [ ]:
###

Вывести список городов, вторая буква в названии которых - 'я'. Для указанных городов вывести названия регионов и федеральных округов.

In [ ]:
sql = '''
    SELECT
        t.city
        ,t.region
        ,t.federal_district
    FROM
        city AS t
    WHERE
        t.city LIKE '_я%'
    ORDER BY
        t.region ASC
    LIMIT 10;
'''

In [ ]:
select(sql)

,city,region,federal_district
0,Няндома,Архангельская,Северо-Западный
1,Дятьково,Брянская,Центральный
2,Кяхта,Бурятия,Дальневосточный
3,Вязники,Владимирская,Центральный
4,Вятские Поляны,Кировская,Приволжский
5,Уяр,Красноярский,Сибирский
6,Сясьстрой,Ленинградская,Северо-Западный
7,Ряжск,Рязанская,Центральный
8,Рязань,Рязанская,Центральный
9,Вязьма,Смоленская,Центральный


In [ ]:
###

Вывести список городов, у которых первая буква 'Е', а вторая не является буквой 'а', 'о', 'у', 'и', 'л'. Также выведите название соответствующего региона и федерального округа.

In [ ]:
sql = '''
    SELECT
        t.city
        ,t.region
        ,t.federal_district
    FROM
        city AS t
    WHERE
        SUBSTR(t.city, 1, 1) = 'Е'
        AND SUBSTR(t.city, 2, 1) NOT IN ('а', 'о', 'у', 'и', 'л')
    ORDER BY
        t.region
    LIMIT 10;
'''

In [ ]:
select(sql)

,city,region,federal_district
0,Ермолино,Калужская,Центральный
1,Емва,Коми,Северо-Западный
2,Ейск,Краснодарский,Южный
3,Енисейск,Красноярский,Сибирский
4,Евпатория,Крым,Южный
5,Егорьевск,Московская,Центральный
6,Ершов,Саратовская,Приволжский
7,Екатеринбург,Свердловская,Уральский
8,Ессентуки,Ставропольский,Северо-Кавказский
9,Ефремов,Тульская,Центральный


In [ ]:
###

Провести арифметические операции с датами.

In [ ]:
sql = '''
    --вывести текушую дату
    SELECT DATE('now')
'''

In [ ]:
select(sql)

,DATE('now')
0,2024-02-09


In [ ]:
sql = '''
    --вывести дату через три дня
    SELECT DATE('now', '3 days')
'''

In [ ]:
select(sql)

,"DATE('now', '3 days')"
0,2024-02-12


In [ ]:
sql = '''
    --вывести дату, которая была 11 дней назад
    SELECT DATE('now', '-11 days')
'''

In [ ]:
select(sql)

,"DATE('now', '-11 days')"
0,2024-01-29


In [ ]:
sql = '''
    --вывести дату, которая была 4 месяца назад от 29 декабря 2023
    SELECT DATE('2023-12-29', '-4 months')
'''

In [ ]:
select(sql)

,"DATE('2023-12-29', '-4 months')"
0,2023-08-29


In [ ]:
sql = '''
    --вывести дату, которая наступит через 3 дня после 19 февраля 2024
    SELECT DATE('2024-02-19', '3 days')
'''

In [ ]:
select(sql)

,"DATE('2024-02-19', '3 days')"
0,2024-02-22


In [ ]:
sql = '''
    --вывести дату, соответствующую четвертому четвергу января 2024
    SELECT DATE(
    '2024-01-01', '21 days', 'weekday 4') AS thyrsday_4th;
'''

In [ ]:
select(sql)

,thyrsday_4th
0,2024-01-25


In [ ]:
###

Рассчитать количество дней между двумя датами.

In [ ]:
sql = '''
    SELECT
    CAST(JULIANDAY('2024-02-06') - JULIANDAY('2024-01-05') AS INT)
        AS days_number;
'''

In [ ]:
select(sql)

,days_number
0,32
